In [1]:
from dash import Dash, dash_table, dcc, html, Output, Input
import pandas as pd
import plotly.graph_objs as go
import requests

In [2]:
# # you can get the chain list straihgt from the API
# url = 'https://api.llama.fi/v2/chains'
# response = requests.get(url)
# if response.status_code == 200: # data was fetched successfully
#     data = response.json()
#     df = pd.DataFrame(data)
# else:
#     raise Exception('Error fetching data from DeFiLlama API')

# dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
# dex_all_chains_response = requests.get(dex_all_chains_url)

# dex_all_chains_data = dex_all_chains_response.json()
# dex_all_chains = set(dex_all_chains_data.get('allChains', []))


# fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
# fees_all_chains_response = requests.get(fees_all_chains_url)

# fees_all_chains_data = fees_all_chains_response.json()
# fees_all_chains = set(fees_all_chains_data.get('allChains', []))

# common_chains = dex_all_chains.intersection(fees_all_chains)

# df = df[['name', 'tvl']]
# df = df[df['name'].isin(common_chains)]

# # Sort the data to get the top 10 chains by TVL
# df = df.sort_values(by='tvl', ascending=False).head(10)

In [3]:
def fetch_historical_data(chain_name, metric):
    if metric == 'tvl':
        url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
            df['date'] = pd.to_datetime(df['date'], unit='s')
            print(df.tail(120))
            return df[['date', metric]].tail(120)  # Get the last 120 days of the specified metric
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'volume':
        url = (f'https://api.llama.fi/overview/dexs/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyVolume')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            print(df.tail(120))
            return df[['date', metric]].tail(120)
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'fee':
        url = (f'https://api.llama.fi/overview/fees/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyFees')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            print(df.tail(120))
            return df[['date', metric]].tail(120)
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
    else:
        raise ValueError('Invalid metric specified')
    

In [ ]:
# Or you can get it thru functions
def fetch_data():
    url = 'https://api.llama.fi/v2/chains'
    response = requests.get(url)
    if response.status_code == 200: # data was fetched successfully
        data = response.json()
        return pd.DataFrame(data)
    else:
        raise Exception('Error fetching data from DeFiLlama API')


def prepare_data(df):
    url = 'https://api.llama.fi/v2/chains'
    response = requests.get(url)
    if response.status_code == 200: # data was fetched successfully
        data = response.json()
        df = pd.DataFrame(data)
    else:
        raise Exception('Error fetching data from DeFiLlama API')

    dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
    dex_all_chains_response = requests.get(dex_all_chains_url)

    dex_all_chains_data = dex_all_chains_response.json()
    dex_all_chains = set(dex_all_chains_data.get('allChains', []))


    fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
    fees_all_chains_response = requests.get(fees_all_chains_url)

    fees_all_chains_data = fees_all_chains_response.json()
    fees_all_chains = set(fees_all_chains_data.get('allChains', []))


    common_chains = dex_all_chains.intersection(fees_all_chains)

    df = df[['name', 'tvl']]
    df = df[df['name'].isin(common_chains)]

    # Sort the data to get the top 10 chains by TVL
    df = df.sort_values(by='tvl', ascending=False).head(10)
    return df


In [5]:
# Step 3: Create the Dash app and the layout
def create_dashboard(df):
    app = Dash(__name__)
    app.layout = html.Div([
        html.H1('DeFi Chains Dashboard'),
        dcc.RadioItems(
            options=[
                {'label': 'Total Value Locked (TVL)', 'value': 'tvl'},
                {'label': 'Volume', 'value': 'volume'},
                {'label': 'Fees', 'value': 'fee'}
            ],
            id='metric-dropdown',
            value='tvl'
            
        ),
        dcc.Graph(id='historical-chart'),
        dash_table.DataTable(
            id='data-table',
            columns=[{'name': col, 'id': col} for col in df.columns],  # Define the columns of the DataTable
            data=df.to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'}
        )
    ])


    @app.callback(
        [Output(component_id='historical-chart', component_property='figure'),
         Output(component_id='data-table', component_property='data')],
        Input(component_id='metric-dropdown', component_property='value')
    )
    def update(selected_metric):
        # Line Chart for historical data of top 10 chains
        historical_fig = go.Figure()
        updated_data = []
        for chain_name in df['name']:
            historical_data_df = fetch_historical_data(chain_name, selected_metric)
            historical_fig.add_trace(go.Scatter(
                x=historical_data_df['date'],
                y=historical_data_df[selected_metric],
                mode='lines',
                name=chain_name
            ))
            # Get the latest value of the selected metric for each chain
            latest_value = historical_data_df[selected_metric].iloc[-1]
            updated_data.append({
                'name': chain_name,
                selected_metric: latest_value
            })
        # Update the layout of the chart
        historical_fig.update_layout(
            title=f'Historical {selected_metric.upper()} of Top 10 Chains (Last 120 Days)',
            xaxis_title='Date',
            yaxis_title=selected_metric.capitalize()
        )
        
        return historical_fig, updated_data

    return app

In [ ]:
if __name__ == '__main__':
    df = fetch_data()
    df = prepare_data(df)
    app = create_dashboard(df)
    app.run_server(debug=True, use_reloader=False)  # Disable reloader to prevent multiple fetches of data

           date           tvl
2287 2024-07-31  5.944718e+10
2288 2024-08-01  5.755419e+10
2289 2024-08-02  5.700088e+10
2290 2024-08-03  5.432958e+10
2291 2024-08-04  5.309768e+10
...         ...           ...
2402 2024-11-23  6.475547e+10
2403 2024-11-24  6.493562e+10
2404 2024-11-25  6.693231e+10
2405 2024-11-26  6.436024e+10
2406 2024-11-27  6.474774e+10

[120 rows x 2 columns]
           date           tvl
1231 2024-07-31  5.310710e+09
1232 2024-08-01  5.032012e+09
1233 2024-08-02  4.950015e+09
1234 2024-08-03  4.686285e+09
1235 2024-08-04  4.505841e+09
...         ...           ...
1346 2024-11-23  9.084357e+09
1347 2024-11-24  9.059608e+09
1348 2024-11-25  9.136232e+09
1349 2024-11-26  8.751647e+09
1350 2024-11-27  8.758267e+09

[120 rows x 2 columns]
           date           tvl
1580 2024-07-31  8.250840e+09
1581 2024-08-01  8.140416e+09
1582 2024-08-02  8.074799e+09
1583 2024-08-03  7.699065e+09
1584 2024-08-04  7.798851e+09
...         ...           ...
1695 2024-11-23  8.212